In [1]:
import math
import numpy as np
import sys

In [2]:
#PREPARING THE DATASET
inp = np.random.randn(20, 32, 32,1)  #Dataset having 20 examples
Y=np.random.randint(9,size=20)   #True Labels
b = np.zeros((20,10))            #One-hot Encoding of True Labels
b[np.arange(20), Y] = 1
Y=b                              #One-hot encoded labels

W1 = np.random.randn(3, 3,1, 4)
W2 =np.random.randn(1024,10)
b1 = np.random.randn(1, 1,1, 4)
b2=np.random.randn(1,10)

stride = 2

In [3]:
def conv_single(a_slice, W, b):
    # Element-wise product between a_slice and W. Add bias.
    s = np.multiply(a_slice, W) + b
    # Sum over all entries of the volume s
    Z = np.sum(s)
    return Z

In [21]:
def conv_forward(inp, W, b, stride, m):
    
    # Retrieve dimensions from A_prev's shape 
    (m, n_H_prev, n_W_prev, n_C_prev) = inp.shape
    
    # Retrieve dimensions from W's shape 
    (f, f, n_C_prev, n_C) = W.shape
    
    pad = 0
    # Computing the dimensions of the CONV output volume using the formula given.
    n_H = math.ceil((n_H_prev - f + (2 * pad)) / stride) + 1
    n_W = math.ceil((n_W_prev - f + (2 * pad)) / stride) + 1
    
    
    # Initialize the output volume Z with zeros.
    Z = np.zeros((m, n_H, n_W, n_C))
    
    
    for i in range(m):                                 # loop over the batch of training examples
        inp_prev= inp[i]                               # Select ith training example
        for h in range(n_H):                           # loop over vertical axis of the output volume
            for w in range(n_W):                       # loop over horizontal axis of the output volume
                for c in range(n_C):                   # loop over channels (= #filters) of the output volume
                                                       # Find the corners of the current "slice"
                    vert_start = h * stride
                    vert_end = vert_start + f
                    horiz_start = w * stride
                    horiz_end = horiz_start + f
                    
                    
                    if (w == (n_W - 1) and h!= (n_H-1)):
                        inp_slice = inp_prev[vert_start:vert_end, horiz_start: horiz_end - 1, :]
                        tmp = W[:,0:2,:]
                        tmp = tmp[...,c]
                        Z[i, h, w, c] = conv_single(inp_slice, tmp, b[...,c])
                    
                    elif ( h == ((n_H - 1))):
                        if (w == (n_W - 1)):
                            inp_slice = inp_prev[vert_start:vert_end-1, horiz_start: horiz_end-1, :]
                            tmp = W[0:2,0:2,:]
                            tmp = tmp[..., c]
                            Z[i, h, w, c] = conv_single(inp_slice, tmp, b[...,c])
                        else:
                            inp_slice = inp_prev[vert_start:vert_end-1, horiz_start: horiz_end, :]
                            tmp = W[0:2,:,:]
                            tmp = tmp[...,c]
                            Z[i, h, w, c] = conv_single(inp_slice, tmp, b[...,c])
                    
                    else:
                        inp_slice = inp_prev[vert_start:vert_end, horiz_start:horiz_end, :]
                        Z[i, h, w, c] = conv_single(inp_slice, W[...,c], b[...,c])
    
    assert(Z.shape == (m, n_H, n_W, n_C))
    
    # Save information in "cache" for the backprop
    cache = (inp, W, b, stride)
    
    return Z, cache

In [22]:
def relu(Z):
    A =  np.maximum(Z, 0)
    cache = Z
    return A, cache

In [23]:
def fc_forward(A, W2, b2):
    Z = np.dot(A,W2) + b2
    cache = (A, W2, b2)
    
    return Z, cache

In [24]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [25]:
def forward_model(inp, W1, W2,b1, b2, stride):
    caches = []
    Z, cache = conv_forward(inp, W1, b1, stride,20)
    caches.append(tuple((cache)))
    Z = Z.reshape(20, -1)
    A, cache = relu(Z)
    caches.append(cache)
    A, cache = fc_forward(A, W2, b2)
    caches.append(tuple((cache)))
    AL=softmax(A)
    output=np.argmax(AL,axis=1)
    
    return AL,caches,output

In [26]:
#Testing the forward model.
AL,caches,output=forward_model(inp,W1,W2,b1,b2, 2)


In [38]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[0]
    dW = np.dot(cache[0].T, dZ) / m
    db = np.squeeze(np.sum(dZ, axis=0, keepdims=True)) / m
    db.reshape(1,10)
    dA_prev = np.dot(dZ, cache[1].T)
    
    return dA_prev, dW, db

In [35]:
def softmax_backward(AL, dAL):
    dZ = np.zeros((20,10))                #initializing dZ to correct shape
    for i in range(10):
        a = AL[:, i]
        dz = np.exp(a)*(np.sum(np.exp(a), axis = 0) - np.exp(a[0]))/(np.square(np.sum(np.exp(a), axis = 0)))  #calculating derivative of softmax
        dZ[:, i] = dz
    dZ = np.multiply(dZ, dAL)            #dZ = dA * g'(z)
    return dZ

In [29]:
def relu_backward(dA, cache):
   
    
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    
    # When z <= 0, dZ should be also zero
    dZ[Z <= 0] = 0
    
    return dZ

In [30]:
def conv_backward(dZ, cache):
    
    # Retrieve information from "cache"
    (A_prev, W, b, s) = cache
    
    # Retrieve dimensions from A_prev's shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve dimensions from W's shape
    (f, f, n_C_prev, n_C) = W.shape
    
    
    
    # Retrieve dimensions from dZ's shape
    (m, n_H, n_W, n_C) = dZ.shape
    
    # Initialize dA_prev, dW, db with the correct shapes
    dA_prev = np.zeros((m, n_H_prev, n_W_prev, n_C_prev))                           
    dW = np.zeros((f, f, n_C_prev, n_C))
    db = np.zeros((1, 1, 1, n_C))

    
    for i in range(m):                       # loop over the training examples
        a_prev = A_prev[i]                   # select ith training example from A_prev_pad and dA_prev_pad
        da_prev = dA_prev[i]
        
        for h in range(n_H):                   # loop over vertical axis of the output volume
            for w in range(n_W):               # loop over horizontal axis of the output volume
                for c in range(n_C):           # loop over the channels of the output volume
                    
                    # Find the corners of the current "slice"
                    vert_start = h
                    vert_end = vert_start + f
                    horiz_start = w
                    horiz_end = horiz_start + f
                    
                    # Use the corners to define the slice from a_prev_pad
                    a_slice = a_prev[vert_start:vert_end, horiz_start:horiz_end, :]

                    # Update gradients for the window and the filter's parameters using the code formulas given above
                    da_prev[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
                    dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
                    db[:,:,:,c] += dZ[i, h, w, c]
                    
        
    
    return dA_prev, dW, db

In [36]:
def backward_model(AL, Y, cache):
    grads = {}
    m = AL.shape[0]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    dAL = dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    current_cache = cache[-1]
    grads["dA2"], grads["dW2"], grads["db2"] = linear_backward(softmax_backward(AL,dAL), current_cache)
    dZ2=relu_backward(grads["dA2"],cache[-2])
    dZ2=dZ2.reshape((20,16,16,4))
    grads["dA1"],grads["dW1"],grads["db1"]=conv_backward(dZ2,cache[-3])
    
    return grads

In [32]:
def update_parameters(W1,b1,W2,b2, grads, lr=0.1):
    W1=W1-lr*grads["dW1"]
    b1=b1-lr*grads["db1"]
    W2=W2-lr*grads["dW2"]
    b2=b2-lr*grads["db2"]
    return W1,b1,W2,b2

In [39]:
for i in range(100):
    AL,caches,output=forward_model(inp,W1,W2,b1,b2, 2)
    grads=backward_model(AL, Y, caches)
    W1,b1,W2,b2=update_parameters(W1,b1,W2,b2, grads, 0.1)


/home/jamrashi28/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less_equal
  
